In [2]:
import json
import pandas as pd
import requests
from simple_salesforce import Salesforce, SalesforceLogin, SFType

In [3]:
sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm0104",
    security_token="EwVPSL02qXM8XQ2pkUdA13aW",
)

In [5]:
results = sf.query_all(
    """
   select Id, IdentificationId__pc from Account 
    """
)
df = pd.DataFrame(results["records"])
df.shape
Pacientes_SF = df[["Id", "IdentificationId__pc"]]
Pacientes_SF.shape

(1562401, 2)

In [64]:
# Recuperar empresas ID de salesforce
results = sf.query_all(
    """
   select Id, Name, Holding__c ,Pa_s_del_Paciente__c,Campa_a__c,AccountNumber,Id_Empresa__c,Empleador__c, ParentId, IsPersonAccount,poliza__c  from account where ispersonaccount = false 
    """
)
df = pd.DataFrame(results["records"])
df.shape
Empresas_account_SF = df[["Id", "Name", "Pa_s_del_Paciente__c", "Campa_a__c","AccountNumber",'Id_Empresa__c','Empleador__c','ParentId','IsPersonAccount','poliza__c']]
Empresas_account_SF.head()



,Id,Name,Pa_s_del_Paciente__c,Campa_a__c,AccountNumber,Id_Empresa__c,Empleador__c,ParentId,IsPersonAccount,poliza__c
0,0018c00002eTcCQAA0,Vida Camara Holding Vida Camara Cintegral Sist...,a1U8c000005dAtnEAE,Vida Camara,None,EMP879,None,0018c00002axQWuAAM,False,None
1,0018c00002eTcCRAA0,Vida Camara Holding Vida Camara Civil Agro S.A.,a1U8c000005dAtnEAE,Vida Camara,None,EMP891,None,0018c00002axQWuAAM,False,None
2,0018c00002eTcCSAA0,Vida Camara Holding Vida Camara Civilplas S.A.,a1U8c000005dAtnEAE,Vida Camara,None,EMP893,None,0018c00002axQWuAAM,False,None
3,0018c00002eTcCTAA0,Vida Camara Holding Vida Camara Cj Traffic Chi...,a1U8c000005dAtnEAE,Vida Camara,None,EMP894,None,0018c00002axQWuAAM,False,None
4,0018c00002eTcCUAA0,Vida Camara Holding Vida Camara Clinda Ltda.,a1U8c000005dAtnEAE,Vida Camara,None,EMP895,None,0018c00002axQWuAAM,False,None


In [65]:
#traer empresas de accouint para unir con empresas sf
results = sf.query_all(
    """
   select id,Name,Nombre_Empresa__c,Empresa__c, Empresa_Padre__c from Programas_por_Empresas__c
    """
)
df = pd.DataFrame(results["records"])
df.shape
Empresas_SF = df[["Id", "Nombre_Empresa__c", "Name", "Empresa__c","Empresa_Padre__c"]]
Empresas_SF.head()


,Id,Nombre_Empresa__c,Name,Empresa__c,Empresa_Padre__c
0,a1W8c000007vDN8EAM,Metlife Holding Metlife Laboratorio Maver Ltda.,A-0000,0018c00002axQXrAAM,MetLife
1,a1W8c000007vDN9EAM,Aguas Del Valle S.A.,A-0001,0018c00002axQXsAAM,MetLife
2,a1W8c000007vDNAEA2,Metlife Holding Metlife Berliam Spa.,A-0002,0018c00002axQXtAAM,MetLife
3,a1W8c000007vDNBEA2,Besalco Maquinarias S.A.,A-0003,0018c00002axQXuAAM,MetLife
4,a1W8c000007vDNCEA2,Carlos Escarate Y Cia Ltda.,A-0004,0018c00002axQXvAAM,MetLife


In [66]:
empresas = pd.merge(
    left=Empresas_account_SF,
    right=Empresas_SF,
    how="left",
    left_on="Id",
    right_on="Empresa__c",
)
empresas.head()

,Id_x,Name_x,Pa_s_del_Paciente__c,Campa_a__c,AccountNumber,Id_Empresa__c,Empleador__c,ParentId,IsPersonAccount,poliza__c,Id_y,Nombre_Empresa__c,Name_y,Empresa__c,Empresa_Padre__c
0,0018c00002eTcCQAA0,Vida Camara Holding Vida Camara Cintegral Sist...,a1U8c000005dAtnEAE,Vida Camara,None,EMP879,None,0018c00002axQWuAAM,False,None,a1W8c0000083l2BEAQ,Vida Camara Holding Vida Camara Cintegral Sist...,A-2357,0018c00002eTcCQAA0,Vida Camara
1,0018c00002eTcCRAA0,Vida Camara Holding Vida Camara Civil Agro S.A.,a1U8c000005dAtnEAE,Vida Camara,None,EMP891,None,0018c00002axQWuAAM,False,None,a1W8c0000083l2CEAQ,Vida Camara Holding Vida Camara Civil Agro S.A.,A-2358,0018c00002eTcCRAA0,Vida Camara
2,0018c00002eTcCSAA0,Vida Camara Holding Vida Camara Civilplas S.A.,a1U8c000005dAtnEAE,Vida Camara,None,EMP893,None,0018c00002axQWuAAM,False,None,a1W8c0000083l2DEAQ,Vida Camara Holding Vida Camara Civilplas S.A.,A-2359,0018c00002eTcCSAA0,Vida Camara
3,0018c00002eTcCTAA0,Vida Camara Holding Vida Camara Cj Traffic Chi...,a1U8c000005dAtnEAE,Vida Camara,None,EMP894,None,0018c00002axQWuAAM,False,None,a1W8c0000083l2EEAQ,Vida Camara Holding Vida Camara Cj Traffic Chi...,A-2360,0018c00002eTcCTAA0,Vida Camara
4,0018c00002eTcCUAA0,Vida Camara Holding Vida Camara Clinda Ltda.,a1U8c000005dAtnEAE,Vida Camara,None,EMP895,None,0018c00002axQWuAAM,False,None,a1W8c0000083l2FEAQ,Vida Camara Holding Vida Camara Clinda Ltda.,A-2361,0018c00002eTcCUAA0,Vida Camara


In [67]:
empresas.shape

(7931, 15)

In [69]:
empresas.to_excel("F:/Empresas SF/reportes/emresas_.xlsx",index=False)

In [70]:

# Relacion pacientes y empresas
results = sf.query_all(
    """
  select Id,Paciente__c,Programas_por_Empresas__c from Paciente_Programas__c 
    """
)
df = pd.DataFrame(results)
df.shape
Pacientes_empresas = pd.DataFrame(results["records"])
Pacientes_empresas = Pacientes_empresas[
    ["Paciente__c", "Programas_por_Empresas__c"]
]
Pacientes_empresas.head()

,Paciente__c,Programas_por_Empresas__c
0,0018c00002amLbpAAE,a1W8c000007vDPvEAM
1,0018c00002amLbqAAE,a1W8c000007vDSpEAM
2,0018c00002amLbrAAE,a1W8c000007vDSpEAM
3,0018c00002amLbsAAE,a1W8c000007vDSpEAM
4,0018c00002amLc2AAE,a1W8c000007vDSpEAM


In [71]:
#une Pacientes y el que hace de union Pacientes_empresas
pacientes_y_union = pd.merge(
    left=Pacientes_SF,
    right=Pacientes_empresas,
    how="left",
    left_on="Id",
    right_on="Paciente__c",
)
pacientes_y_union.head()

,Id,IdentificationId__pc,Paciente__c,Programas_por_Empresas__c
0,0018c00002amLTMAA2,173805004,0018c00002amLTMAA2,a1W8c0000083icPEAQ
1,0018c00002amLUGAA2,153338949,NaN,NaN
2,0018c00002amLUHAA2,177735213,0018c00002amLUHAA2,a1W8c0000083j8wEAA
3,0018c00002amLUIAA2,169332924,0018c00002amLUIAA2,a1W8c0000083laZEAQ
4,0018c00002amLUJAA2,172957692,0018c00002amLUJAA2,a1W8c0000083iWhEAI


In [72]:
pacientes_y_union.shape

(1671398, 4)

In [73]:
pacientes_y_empresas_final = pd.merge(
    left=pacientes_y_union,
    right=Empresas_SF,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Id",
)
pacientes_y_empresas_final.head()

,Id_x,IdentificationId__pc,Paciente__c,Programas_por_Empresas__c,Id_y,Nombre_Empresa__c,Name,Empresa__c,Empresa_Padre__c
0,0018c00002amLTMAA2,173805004,0018c00002amLTMAA2,a1W8c0000083icPEAQ,a1W8c0000083icPEAQ,Vulco S.A.,A-1075,0018c00002cXJZeAAO,MetLife
1,0018c00002amLUGAA2,153338949,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0018c00002amLUHAA2,177735213,0018c00002amLUHAA2,a1W8c0000083j8wEAA,a1W8c0000083j8wEAA,Farmacias Cruz Verde,A-2322,0018c00002cXclVAAS,Farmacias Cruz Verde
3,0018c00002amLUIAA2,169332924,0018c00002amLUIAA2,a1W8c0000083laZEAQ,a1W8c0000083laZEAQ,EY - EY Audit SpA,A-4489,0018c00002eTboVAAS,EY
4,0018c00002amLUJAA2,172957692,0018c00002amLUJAA2,a1W8c0000083iWhEAI,a1W8c0000083iWhEAI,Minera Abra Holding Minera Abra Minera Abra,A-0721,0018c00002cXJTrAAO,Minera Abra


In [74]:
pacientes_y_empresas_final.shape

(1671398, 9)

In [75]:
nuevos = ("F:/Empresas SF/reportes/activos.xlsx")
Pacientes_Archivo = pd.read_excel(nuevos)
# OJO se copia rut1 a rut le saca el guoion
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut1"].str.replace("-", "")

# aca se trabaja con el nuevo rut desde rut1 ojooooooo
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.upper()
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.strip()
Pacientes_Archivo.head()

,Nombre de Empleador,Rut de trabajador,DV,rut1,Nombre de trabajador,nombres,apellidos,rut
0,COPEFRUT SA,20261991,6,20261991-6,YOSTIN JOAQUIN LEON MILLA,YOSTIN JOAQUIN,LEON MILLA,202619916
1,COPEFRUT SA,18407124,K,18407124-K,PAILEMILLA SARMIENTO LEONARDO ANDRES,PAILEMILLA SARMIENTO,LEONARDO ANDRES,18407124K
2,COPEFRUT SA,17910195,5,17910195-5,CUEVAS CORREA PATRICIO ANDRES,CUEVAS CORREA,PATRICIO ANDRES,179101955
3,COPEFRUT SA,14300595,K,14300595-K,BRENDA MONSERRAT CAMPOS ORTIZ,BRENDA MONSERRAT,CAMPOS ORTIZ,14300595K
4,COPEFRUT SA,19192035,K,19192035-K,JOSE FERNANDO VERGARA BRITO,JOSE FERNANDO,VERGARA BRITO,19192035K


In [76]:
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(",", "")
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.title()
#Pacientes_Archivo["empresa"] = Pacientes_Archivo["empresa"].str.title()
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(",", "")
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.title()
Pacientes_SF["IdentificationId__pc"] = Pacientes_SF["IdentificationId__pc"].str.upper()
Pacientes_SF["IdentificationId__pc"] = Pacientes_SF["IdentificationId__pc"].astype(str)
Pacientes_SF["IdentificationId__pc"] = Pacientes_SF["IdentificationId__pc"].str.strip()
Pacientes_SF["IdentificationId__pc"] = Pacientes_SF["IdentificationId__pc"].str.replace(
    "-", ""
)


Pacientes_Archivo["RutEstaoNo"] = Pacientes_Archivo["rut"].isin(
    Pacientes_SF["IdentificationId__pc"])

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_10208\1296246679.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_10208\1296246679.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")


In [77]:
#Pacientes_Archivo.head(20)

In [78]:
pacientes_y_empresas_final["IdentificationId__pc"] = pacientes_y_empresas_final["IdentificationId__pc"].str.upper()
pacientes_y_empresas_final["IdentificationId__pc"] = pacientes_y_empresas_final["IdentificationId__pc"].astype(str)
pacientes_y_empresas_final["IdentificationId__pc"] = pacientes_y_empresas_final["IdentificationId__pc"].str.strip()
pacientes_y_empresas_final["IdentificationId__pc"] = pacientes_y_empresas_final["IdentificationId__pc"].str.replace(
    "-", ""
)
pacientes_y_empresas_final["IdentificationId__pc"] = pacientes_y_empresas_final["IdentificationId__pc"].str.replace(
    ".", ""
)

Pacientes_con_ID = pd.merge(
    left=Pacientes_Archivo,
    right=pacientes_y_empresas_final,
    how="left",
    left_on="rut",
    right_on="IdentificationId__pc",
)
Pacientes_con_ID.head()


C:\Users\jmartinez\AppData\Local\Temp\ipykernel_10208\456707719.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  pacientes_y_empresas_final["IdentificationId__pc"] = pacientes_y_empresas_final["IdentificationId__pc"].str.replace(


,Nombre de Empleador,Rut de trabajador,DV,rut1,Nombre de trabajador,nombres,apellidos,rut,RutEstaoNo,Id_x,IdentificationId__pc,Paciente__c,Programas_por_Empresas__c,Id_y,Nombre_Empresa__c,Name,Empresa__c,Empresa_Padre__c
0,COPEFRUT SA,20261991,6,20261991-6,YOSTIN JOAQUIN LEON MILLA,Yostin Joaquin,Leon Milla,202619916,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,COPEFRUT SA,18407124,K,18407124-K,PAILEMILLA SARMIENTO LEONARDO ANDRES,Pailemilla Sarmiento,Leonardo Andres,18407124K,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,COPEFRUT SA,17910195,5,17910195-5,CUEVAS CORREA PATRICIO ANDRES,Cuevas Correa,Patricio Andres,179101955,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,COPEFRUT SA,14300595,K,14300595-K,BRENDA MONSERRAT CAMPOS ORTIZ,Brenda Monserrat,Campos Ortiz,14300595K,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,COPEFRUT SA,19192035,K,19192035-K,JOSE FERNANDO VERGARA BRITO,Jose Fernando,Vergara Brito,19192035K,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
Pacientes_con_ID.to_excel(
    "F:/Empresas SF/reportes/Pacientes_y_empresas_nuevas.xlsx", index=False
)


In [80]:
Final_pacientes_y_empresa = Pacientes_con_ID
Subir_pacientes = pd.DataFrame()
# agrupar = empresasqfaltan[empresasqfaltan['si']==False]
Final_pacientes_y_empresa = Final_pacientes_y_empresa[
    Final_pacientes_y_empresa["RutEstaoNo"] == False
]

if ("nombres" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["FirstName"] = Final_pacientes_y_empresa["nombres"]
if ("apellidos" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["LastName"] = Final_pacientes_y_empresa["apellidos"]
if ("rut" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["IdentificationId__pc"] = Final_pacientes_y_empresa["rut"]
if ("email" in Final_pacientes_y_empresa.columns) == True:
    Final_pacientes_y_empresa["email"] = Final_pacientes_y_empresa["email"].astype(str).str.lower().replace("nan","")
    Subir_pacientes["PersonEmail"] = Final_pacientes_y_empresa["email"]
if ("nacimiento" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["PersonBirthdate"] = Final_pacientes_y_empresa["nacimiento"]
if ("sexo" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["HealthCloudGA__Gender__pc"] = Final_pacientes_y_empresa["sexo"]
if ("direccion" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["BillingStreet"] = Final_pacientes_y_empresa["direccion"].str.title().replace("NaN","")
    #Subir_pacientes["BillingStreet"] = Final_pacientes_y_empresa["direccion"]
if ("comuna" in Final_pacientes_y_empresa.columns) == True:
    #Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
    Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"].str.title().replace("NaN","")
    #Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
if ("ciudad" in Final_pacientes_y_empresa.columns) == True:
    #Subir_pacientes["BillingCity"] = Final_pacientes_y_empresa["ciudad"]
    Subir_pacientes["BillingCity"] = Final_pacientes_y_empresa["ciudad"].str.title().replace("NaN","") 
if ("cargo" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["PersonTitle"] = Final_pacientes_y_empresa["cargo"]
if ("division" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["PersonDepartment"] = Final_pacientes_y_empresa["division"]
if ("celular" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["Phone"] = Final_pacientes_y_empresa["celular"]


Subir_pacientes["BillingCountry"] = "Chile"
Subir_pacientes["Identification_Type__c"] = "RUT"
Subir_pacientes["RecordTypeId"] = "0128c000002Byp4AAC"

#Subir_pacientes.PersonEmail = Subir_pacientes.PersonEmail.fillna('')
Subir_pacientes.to_excel(
     "F:/Empresas SF/reportes/account.xlsx", index=False
)
Subir_pacientes.head()


,FirstName,LastName,IdentificationId__pc,BillingCountry,Identification_Type__c,RecordTypeId
0,Yostin Joaquin,Leon Milla,202619916,Chile,RUT,0128c000002Byp4AAC
1,Pailemilla Sarmiento,Leonardo Andres,18407124K,Chile,RUT,0128c000002Byp4AAC
2,Cuevas Correa,Patricio Andres,179101955,Chile,RUT,0128c000002Byp4AAC
3,Brenda Monserrat,Campos Ortiz,14300595K,Chile,RUT,0128c000002Byp4AAC
4,Jose Fernando,Vergara Brito,19192035K,Chile,RUT,0128c000002Byp4AAC


In [6]:
nuevos = ("F:/Empresas SF/reportes/activos2.xlsx")
Pacientes_Archivo = pd.read_excel(nuevos)
# OJO se copia rut1 a rut le saca el guoion
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut1"].str.replace("-", "")
# aca se trabaja con el nuevo rut desde rut1 ojooooooo
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.upper()
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.strip()
Pacientes_Archivo.head()

pacientes_id = pd.merge(
    left=Pacientes_Archivo,
    right=Pacientes_SF,
    how="left",
    left_on="rut",
    right_on="IdentificationId__pc",
)
pacientes_id.head()
pacientes_id.to_excel(
     "F:/Empresas SF/reportes/archivo_id.xlsx", index=False
)